In [2]:
# importing libraries 
import pandas as pd
from sklearn.model_selection import train_test_split
import re       #regular expression is used to search the words in a text
import spacy
import nltk 
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords  #stopwords means those words which doesn't add much value to text context like articles
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Importing dataset

In [9]:
  #useful in creaing dataframs and storing the data in the dataframes
data  = pd.read_csv("https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true",compression='zip')
data.head(2)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184


### Exploratory data analysis

In [10]:
data['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [ ]:
data['Company'].value_counts()

CITIBANK, N.A.                                                    3226
CAPITAL ONE FINANCIAL CORPORATION                                 2711
BANK OF AMERICA, NATIONAL ASSOCIATION                             2580
JPMORGAN CHASE & CO.                                              2409
WELLS FARGO & COMPANY                                             2001
                                                                  ... 
Time Investment Company, Inc.                                        1
Automotive Services Finance, Inc.                                    1
Foxstar Energy Resources LLC DBA Federal Student Loans Council       1
Uplift, Inc                                                          1
Keystone Credit Services LLC                                         1
Name: Company, Length: 2197, dtype: int64

In [11]:
# considering usefull data columns (Product,company,Consumer complaint narrative)

complaint_data = data[['Product','Company','Consumer complaint narrative']].rename(columns={'Consumer complaint narrative':'Complaint'})

pd.set_option('display.max_colwidth', -1)
complaint_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,Product,Company,Complaint
0,Vehicle loan or lease,TRUIST FINANCIAL CORPORATION,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account."
1,Debt collection,CURO Intermediate Holdings,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back."
2,Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times."
3,Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that."
4,Debt collection,"Merchants and Professional Bureau, Inc.","This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. 

In [ ]:
# more exploration on dataset
print(complaint_data.isnull().count())

Product      57453
Company      57453
Complaint    57453
dtype: int64


In [ ]:
print(complaint_data.dtypes)

Product      object
Company      object
Complaint    object
dtype: object


### **Data Preprocessing**

In [12]:
# data set split for further analysis 

train,test = train_test_split(complaint_data,test_size=0.3,random_state=42)

In [13]:
train.shape

(40217, 3)

In [14]:

train['Complaint_processed'] = train['Complaint'].map(lambda x: re.sub('[,\.!?]', '', x))  #(˄ means exclusion of everything which is not alphabets and replacing it with null space)

In [15]:
train['Complaint_processed'] = train['Complaint_processed'].map(lambda x: x.lower())   #converting all the letters into lowercase
train['Complaint_processed'].head()   

42027    after many weeks of receiving hourly phone calls from navient i sent them a letter instructing them only to contact me by mail per the fair debt collection practices act and the calls stopped although they have emailed me a couple of times today xx/xx/19 the mother of a friend from middle school let me know that navient had called her and left a message for me she is not listed as an emergency or secondary contact in any of my paperwork ; navient should not have her number unless they dig it up themselves this is a huge breach of privacy and a clear violation of the fdcpa given that i had instructed them to only contact me by phone and that they contacted someone completely unrelated to me and told them they were looking for me regarding student loan debt per the fdcpa : " a debt collector may not communicate in connection with the collection of any debt with any person other than the consumer his attorney a consumer reporting agency if otherwise permitted by law the creditor 

In [ ]:
!pip install nltk   #natural language toolkit 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
stop_words = (stopwords.words('english'))

stemmer = PorterStemmer()
stemmer = nltk.stem.SnowballStemmer('english')

## Tokenisation

In [17]:

def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+', '', word.strip('Xx/'))) > 3) ] 
  #  tokens = map(str.lower, tokens)
   stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
   return stems


docs = train.Complaint_processed.values.tolist()
data_words = [tokenize(doc) for doc in docs]

In [18]:
print(len(data_words))

data_words[:2]

40217


[['mani',
  'week',
  'receiv',
  'hour',
  'phone',
  'call',
  'navient',
  'sent',
  'letter',
  'instruct',
  'contact',
  'mail',
  'fair',
  'debt',
  'collect',
  'practic',
  'call',
  'stop',
  'although',
  'email',
  'coupl',
  'time',
  'today',
  'mother',
  'friend',
  'middl',
  'school',
  'know',
  'navient',
  'call',
  'left',
  'messag',
  'list',
  'emerg',
  'secondari',
  'contact',
  'paperwork',
  'navient',
  'number',
  'unless',
  'huge',
  'breach',
  'privaci',
  'clear',
  'violat',
  'fdcpa',
  'given',
  'instruct',
  'contact',
  'phone',
  'contact',
  'someon',
  'complet',
  'unrel',
  'told',
  'look',
  'regard',
  'student',
  'loan',
  'debt',
  'fdcpa',
  'debt',
  'collector',
  'communic',
  'connect',
  'collect',
  'debt',
  'person',
  'consum',
  'attorney',
  'consum',
  'report',
  'agenc',
  'otherwis',
  'permit',
  'creditor',
  'attorney',
  'creditor',
  'attorney',
  'debt',
  'collector',
  'navient',
  'clear',
  'violat',
  'pe

## Convert Text into Numerical Representation


In [19]:
# In Natural language Processing there are many vectorizer which can convert text tokens to numerical tokens here we will try only two method 
# 1 - TF-IFD vectorizer    
# 2 - Count Vectorizer
# You can  find more about them in Document file

tf_ifd  = TfidfVectorizer(tokenizer=tokenize, max_df = 0.75, min_df=50 , max_features=10000,use_idf= False,  lowercase=False )

# Converting text into numerical representation
cv = CountVectorizer(tokenizer=tokenize, max_df = 0.75, min_df=50 , max_features=10000, lowercase=False)

In [20]:
tf_vector = tf_ifd.fit_transform(train.Complaint_processed)
vc_vectore = cv.fit_transform(train.Complaint_processed)

In [21]:
print("tf_vector matrx",tf_vector.A)
tf_vector.shape

tf_vector matrx [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(40217, 2602)

In [22]:
print("count vectorizer matrix ",vc_vectore.A)
vc_vectore.shape

count vectorizer matrix  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(40217, 2602)

In [23]:
cv_voc = (cv.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
tf_voc = tf_ifd.get_feature_names()

In [25]:
print("length of tf_ifd vector vocabalury is :",len(tf_ifd.get_feature_names()))
print("length of cv vector vocabalury is :",len(cv.get_feature_names()))

length of tf_ifd vector vocabalury is : 2602
length of cv vector vocabalury is : 2602


## LDA Implementation Using Sckit learn

In [ ]:
n_topics = 8
model_lda = LDA(n_components = n_topics, learning_method='online',max_iter = 20,learning_offset=50,n_jobs = -1, random_state = 42)
# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = model_lda.fit_transform(tf_vector)



In [ ]:

# .components_ gives us our topic distribution 
topic_words = model_lda.components_

In [ ]:
X_topics

array([[0.01602825, 0.01602824, 0.016024  , ..., 0.01605815, 0.016024  ,
        0.01602578],
       [0.01112842, 0.01111141, 0.01111128, ..., 0.92211192, 0.01111128,
        0.01114164],
       [0.01640639, 0.0163859 , 0.01636912, ..., 0.01641548, 0.01636912,
        0.01637997],
       ...,
       [0.65803602, 0.02186396, 0.02186366, ..., 0.21065441, 0.02186366,
        0.02187585],
       [0.02040153, 0.02035488, 0.02034854, ..., 0.02041819, 0.02034854,
        0.02036167],
       [0.01949243, 0.01948182, 0.01946629, ..., 0.01949309, 0.01946629,
        0.01952527]])

In [ ]:
topic_words

array([[ 0.12500005,  1.97872592,  0.1260384 , ...,  0.12607867,
         0.12585044,  0.12511835],
       [ 0.12500023,  0.12502719,  0.12500227, ...,  0.12500161,
         0.1250494 ,  0.12501165],
       [ 0.12500133,  0.12500887,  0.12501411, ...,  0.12500958,
         0.12500832,  0.12502112],
       ...,
       [ 0.125     ,  1.28019551,  1.17852976, ...,  2.12105489,
        18.71096697,  1.1796296 ],
       [ 0.12500133,  0.12500887,  0.12501411, ...,  0.12500958,
         0.12500832,  0.12502112],
       [ 0.12500024,  0.12500161,  0.12500255, ...,  0.12500174,
         3.82780958,  0.12500385]])

So, **what X_topics and topic_words represents ?**


*  **X_topics**: represents probability of each topic to be assigned to each document. So, it should be (no. of doc , no. of topics) shape.It is known as Document-topic matrix.
*   **topic_words**: represents probability of particualr word being in particular topic. So, it is (no. of topics, no. of words) shape.It is known as Topic-Word matrix.



In [ ]:
print(topic_words.shape)
print(X_topics.shape)

(8, 2602)
(40217, 8)


In [ ]:
n_top_words = 15
topics = []

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(tf_voc)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    topics.append(topic_words)
    print ("Topic", str(i), topic_words)
topics=np.array(topics)

Topic 0 ['credit' 'payment' 'late' 'account' 'report' 'balanc' 'card' 'interest'
 'month' 'charg' 'statement' 'paid' 'made' 'date']
Topic 1 ['ident' 'theft' 'victim' 'report' 'belong' 'attach' 'pleas' 'debt'
 'affidavit' 'legal' 'result' 'discov' 'monitor' 'other']
Topic 2 ['repositori' 'endless' 'enrich' 'balloon' 'devast' 'deactiv' 'helpless'
 'surround' 'realiti' 'protocol' 'southern' 'rush' 'contravent' 'rebil']
Topic 3 ['payment' 'loan' 'call' 'mortgag' 'would' 'month' 'time' 'compani'
 'receiv' 'told' 'account' 'servic' 'year' 'amount']
Topic 4 ['debt' 'credit' 'report' 'account' 'collect' 'compani' 'inform' 'receiv'
 'letter' 'disput' 'never' 'request' 'provid' 'remov']
Topic 5 ['account' 'call' 'card' 'bank' 'credit' 'charg' 'receiv' 'would' 'check'
 'time' 'told' 'number' 'money' 'back']
Topic 6 ['repositori' 'endless' 'enrich' 'balloon' 'devast' 'deactiv' 'helpless'
 'surround' 'realiti' 'protocol' 'southern' 'rush' 'contravent' 'rebil']
Topic 7 ['account' 'pleas' 'sure' 'cre

In [ ]:
doc_topic = model_lda.transform(tf_vector)  
topic_to_doc = []
# iterating over ever value till the end value
for n in range(doc_topic.shape[0]):
    
    # argmax() gives maximum index value
    topic_doc = doc_topic[n].argmax()
    topic_to_doc.append(topic_doc)

def doc_to_topic(doc_no,topic_to_doc=topic_to_doc):
  print("topic assigned to document ",doc_no," is ",topic_to_doc[doc_no+1]," that is ",topics[topic_to_doc[doc_no+1]])

doc_to_topic(2)

topic assigned to document  2  is  5  that is  ['account' 'call' 'card' 'bank' 'credit' 'charg' 'receiv' 'would' 'check'
 'time' 'told' 'number' 'money' 'back']


In [ ]:
topic_assigned_to_doc = pd.DataFrame(X_topics,columns=['topic0','topic1','topic2','topic3','topic4','topic5','topic6','topic7'])
topic_assigned_to_doc['topic_assigned'] = topic_to_doc

In [ ]:
topic_assigned_to_doc

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic_assigned
0,0.016028,0.016028,0.016024,0.333768,0.570044,0.016058,0.016024,0.016026,4
1,0.011128,0.011111,0.011111,0.011148,0.011136,0.922112,0.011111,0.011142,5
2,0.016406,0.016386,0.016369,0.885249,0.016425,0.016415,0.016369,0.016380,3
3,0.016411,0.016507,0.016389,0.016420,0.016413,0.885026,0.016389,0.016445,5
4,0.047475,0.047542,0.047475,0.047493,0.667529,0.047537,0.047475,0.047475,4
...,...,...,...,...,...,...,...,...,...
40212,0.015152,0.015139,0.015137,0.893944,0.015162,0.015177,0.015137,0.015152,3
40213,0.019786,0.019805,0.019759,0.019789,0.406082,0.474967,0.019759,0.020053,5
40214,0.658036,0.021864,0.021864,0.021918,0.021925,0.210654,0.021864,0.021876,0
40215,0.020402,0.020355,0.020349,0.020407,0.857359,0.020418,0.020349,0.020362,4


In [26]:
from sklearn.cluster import KMeans
km = KMeans(8,init='k-means++',max_iter=20)
km.fit(tf_vector)

KMeans(max_iter=20)

In [30]:
centroids = km.cluster_centers_.argsort()[:,::-1]
for i in range(8):
  print("centroid ", i)
  for ind in centroids[i,:15]:
    print(tf_voc[ind],end=' ')
  print()

centroid  0
credit report account compani collect inform debt remov never disput file receiv request payment time 
centroid  1
payment month account loan call late credit made mortgag would time make receiv bank told 
centroid  2
call told time number phone receiv would back compani account said ask debt inform contact 
centroid  3
card credit charg account call bank receiv would time told disput never purchas payment balanc 
centroid  4
debt collect credit report compani account receiv letter inform valid provid contact request disput call 
centroid  5
ident theft report belong attach victim debt affidavit pleas legal account inform result fraudul file 
centroid  6
loan mortgag receiv time compani would servic payment call month inform bank told state request 
centroid  7
account bank check credit close call open charg receiv money would inform time deposit told 


In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()